In [52]:
import pandas as pd
from fuzzywuzzy import process
from sklearn.preprocessing import MinMaxScaler


In [ ]:
data = pd.read_csv("final_table_florida.csv")
county_info = pd.read_excel("county_info.xlsx")

In [3]:
# Merge in county info# Add in county information for all LEAIDs
cols = ["LEAID", "NMCNTY"]
county_info = county_info[cols]
county_info['LEAID'] = pd.to_numeric(county_info['LEAID'], errors='coerce')

# Get the most common county for each school district
county_info = county_info.groupby('LEAID')['NMCNTY'].agg(lambda x: x.mode().iloc[0])

county_merged = pd.merge(data, county_info, on='LEAID', how='left')
county_merged

,Unnamed: 0,LEA_STATE_NAME,LEAID,LEA_NAME,banned_books,LEA_ENR,SCH_FTETEACH_TOT,SATACT_P,APENR_P,P_EXPULSIONS,TOT_DIS_HB,TOT_RAC_HB,TOT_SEX_HB,NMCNTY
0,0,FLORIDA,1200002,FL VIRTUAL,False,6748,0.008595,7.898637,5.853586,0.000000,NaN,NaN,NaN,Orange County
1,1,FLORIDA,1200030,ALACHUA,False,29737,0.057114,7.694118,8.037125,0.016814,NaN,NaN,NaN,Alachua County
2,2,FLORIDA,1200060,BAKER,False,5044,0.058541,8.068993,0.733545,0.693894,NaN,NaN,NaN,Baker County
3,3,FLORIDA,1200090,BAY,False,28073,0.061955,7.020981,7.929327,0.295658,NaN,NaN,NaN,Bay County
4,4,FLORIDA,1200120,BRADFORD,False,3214,0.053843,6.969508,2.955818,0.311139,NaN,NaN,NaN,Bradford County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,FLORIDA,1202012,FAU LAB SCH,False,2555,0.058603,7.632094,NaN,0.000000,NaN,NaN,NaN,Palm Beach County
69,69,FLORIDA,1202013,FSU LAB SCH,False,2405,0.057838,7.983368,11.018711,0.083160,NaN,NaN,NaN,Leon County
70,70,FLORIDA,1202014,FAMU LAB SCH,False,586,0.073379,8.020478,1.877133,0.000000,NaN,NaN,NaN,Leon County
71,71,FLORIDA,1202015,UF LAB SCH,False,1153,0.052905,14.397225,22.549870,0.000000,NaN,NaN,NaN,Alachua County


In [9]:
def parse_slash(text):
    parts = text.split('/')
    first_part = parts[0].strip()
    return first_part

In [21]:
# Merge in income information
wealth = pd.read_csv("florida_school_district_wealth.csv")
wealth["Median Household Income"] = wealth["Median Household Income"].str.replace("$", "").str.strip()
wealth["Median Household Income"] = wealth["Median Household Income"].str.replace(",", "").str.strip()
wealth["Median Household Income"] = wealth["Median Household Income"].astype(int)

wealth['dist_key'] = wealth['School District'].apply(parse_slash)
wealth["dist_key"] = wealth["dist_key"].str.replace("School District", "").str.strip()
wealth["dist_key"] = wealth["dist_key"].str.upper()
wealth

<ipython-input-21-ccb5b4df9626>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wealth["Median Household Income"] = wealth["Median Household Income"].str.replace("$", "").str.strip()


,Median Household Income,School District,dist_key
0,78815,"St. Johns County School District / 203,402",ST. JOHNS COUNTY
1,69712,"Santa Rosa County School District / 158,240",SANTA ROSA COUNTY
2,68543,"Seminole County School District / 432,135",SEMINOLE COUNTY
3,68504,"Leon County School District / 280,882",LEON COUNTY
4,66470,"Clay County School District / 194,868",CLAY COUNTY
...,...,...,...
62,38388,"Putnam County School District / 73,191",PUTNAM COUNTY
63,36807,"Taylor County School District / 22,690",TAYLOR COUNTY
64,36375,"Madison County School District / 18,901",MADISON COUNTY
65,36371,"Calhoun County School District / 14,657",CALHOUN COUNTY


In [22]:
def find_closest_match(row, column_to_match):
    choices = wealth[column_to_match]
    result = process.extractOne(row['NMCNTY'], choices)
    
    # Check if a match is found and the score is above the threshold (80 in this example)
    if result is not None and result[1] >= 80:
        return result[0]  # Return the match
    else:
        return None  # Return None if no suitable match is found

county_merged['Closest_Match'] = county_merged.apply(find_closest_match, column_to_match='dist_key', axis=1)


In [ ]:
def find_closest_match(row, column_to_match):
    choices = wealth[column_to_match]
    result = process.extractOne(row['NMCNTY'], choices)
    
    # Check if a match is found and the score is above the threshold (80 in this example)
    if result is not None and result[1] >= 80:
        return result[0]  # Return the match
    else:
        return None  # Return None if no suitable match is found

county_merged['Closest_Match'] = county_merged.apply(find_closest_match, column_to_match='dist_key', axis=1)


In [29]:
# Finish merging in median household income

merged_df = pd.merge(county_merged, wealth[["Median Household Income", "dist_key"]], how='left', left_on='Closest_Match', right_on='dist_key')
merged_df["Median Household Income"].sort_values()

merged_df = merged_df.drop(columns=["Unnamed: 0", "dist_key", "Closest_Match"])

In [32]:
merged_df

,LEA_STATE_NAME,LEAID,LEA_NAME,banned_books,LEA_ENR,SCH_FTETEACH_TOT,SATACT_P,APENR_P,P_EXPULSIONS,TOT_DIS_HB,TOT_RAC_HB,TOT_SEX_HB,NMCNTY,Median Household Income
0,FLORIDA,1200002,FL VIRTUAL,False,6748,0.008595,7.898637,5.853586,0.000000,NaN,NaN,NaN,Orange County,55071
1,FLORIDA,1200030,ALACHUA,False,29737,0.057114,7.694118,8.037125,0.016814,NaN,NaN,NaN,Alachua County,57729
2,FLORIDA,1200060,BAKER,False,5044,0.058541,8.068993,0.733545,0.693894,NaN,NaN,NaN,Baker County,52358
3,FLORIDA,1200090,BAY,False,28073,0.061955,7.020981,7.929327,0.295658,NaN,NaN,NaN,Bay County,54430
4,FLORIDA,1200120,BRADFORD,False,3214,0.053843,6.969508,2.955818,0.311139,NaN,NaN,NaN,Bradford County,44411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,FLORIDA,1202012,FAU LAB SCH,False,2555,0.058603,7.632094,NaN,0.000000,NaN,NaN,NaN,Palm Beach County,63493
69,FLORIDA,1202013,FSU LAB SCH,False,2405,0.057838,7.983368,11.018711,0.083160,NaN,NaN,NaN,Leon County,68504
70,FLORIDA,1202014,FAMU LAB SCH,False,586,0.073379,8.020478,1.877133,0.000000,NaN,NaN,NaN,Leon County,68504
71,FLORIDA,1202015,UF LAB SCH,False,1153,0.052905,14.397225,22.549870,0.000000,NaN,NaN,NaN,Alachua County,57729


In [50]:
county_PVI = pd.read_csv("florida_county_PVI.csv")
county_PVI
# Define a regular expression pattern to extract the numerical values
pattern = r'([+-]?\d+)'

# Extract the numeric values and convert to numeric
county_PVI['PVI_parsed'] = pd.to_numeric(county_PVI['PVI'].str.extract(pattern)[0])

# Assign the sign based on the party
county_PVI['PVI_parsed'] *= county_PVI['PVI'].str.contains('Republican').map({True: 1, False: -1})

# Display the resulting DataFrame
county_PVI

,County,PVI,PVI_parsed
0,Alachua County,Democrat +14,-14
1,Baker County,Republican +33,33
2,Bay County,Republican +21,21
3,Bradford County,Republican +25,25
4,Brevard County,Republican +8,8
...,...,...,...
62,Union County,Republican +31,31
63,Volusia County,Republican +6,6
64,Wakulla County,Republican +19,19
65,Walton County,Republican +26,26


In [54]:
final_merged = pd.merge(merged_df, county_PVI, left_on='NMCNTY', right_on='County', how='left')
final_merged = final_merged.drop(columns=["County", "PVI"])

# Scale our PVI
scaler = MinMaxScaler(feature_range=(-1, 1))

# Reshape the column to a 2D array and scale it
final_merged["PVI_parsed"] = scaler.fit_transform(final_merged["PVI_parsed"].values.reshape(-1, 1))
final_merged

,LEA_STATE_NAME,LEAID,LEA_NAME,banned_books,LEA_ENR,SCH_FTETEACH_TOT,SATACT_P,APENR_P,P_EXPULSIONS,TOT_DIS_HB,TOT_RAC_HB,TOT_SEX_HB,NMCNTY,Median Household Income,PVI_parsed
0,FLORIDA,1200002,FL VIRTUAL,False,6748,0.008595,7.898637,5.853586,0.000000,NaN,NaN,NaN,Orange County,55071,-0.766667
1,FLORIDA,1200030,ALACHUA,False,29737,0.057114,7.694118,8.037125,0.016814,NaN,NaN,NaN,Alachua County,57729,-0.766667
2,FLORIDA,1200060,BAKER,False,5044,0.058541,8.068993,0.733545,0.693894,NaN,NaN,NaN,Baker County,52358,0.800000
3,FLORIDA,1200090,BAY,False,28073,0.061955,7.020981,7.929327,0.295658,NaN,NaN,NaN,Bay County,54430,0.400000
4,FLORIDA,1200120,BRADFORD,False,3214,0.053843,6.969508,2.955818,0.311139,NaN,NaN,NaN,Bradford County,44411,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,FLORIDA,1202012,FAU LAB SCH,False,2555,0.058603,7.632094,NaN,0.000000,NaN,NaN,NaN,Palm Beach County,63493,-0.633333
69,FLORIDA,1202013,FSU LAB SCH,False,2405,0.057838,7.983368,11.018711,0.083160,NaN,NaN,NaN,Leon County,68504,-0.833333
70,FLORIDA,1202014,FAMU LAB SCH,False,586,0.073379,8.020478,1.877133,0.000000,NaN,NaN,NaN,Leon County,68504,-0.833333
71,FLORIDA,1202015,UF LAB SCH,False,1153,0.052905,14.397225,22.549870,0.000000,NaN,NaN,NaN,Alachua County,57729,-0.766667


In [55]:
final_merged.to_csv("Final_florida_df.csv")